In [5]:
import requests
import json
import time

session_id = "96baff69c12542ba93060b67a5300b88"

# 1. 먼저 요청 보내기
message = {
    "jsonrpc": "2.0",
    "id": 1,
    "method": "tools/list",
    "params": {}
}

response = requests.post(
    f"http://localhost:6276/messages/?session_id={session_id}",
    json=message
)
print(f"Request Status: {response.status_code}")

# 2. SSE 스트림에서 응답 받기
sse_response = requests.get(
    f"http://localhost:6276/sse?session_id={session_id}",
    stream=True
)

print("SSE Response:")
for line in sse_response.iter_lines():
    if line:
        decoded_line = line.decode('utf-8')
        print(decoded_line)
        
        # 5줄 정도만 확인하고 중단
        if decoded_line.startswith('data:'):
            break

Request Status: 202
SSE Response:
event: endpoint
data: /messages/?session_id=8e4408eef59b4548a4970b381a64df61


In [1]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM

model_id = 'Bllossom/llama-3.2-Korean-Bllossom-3B'

tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    torch_dtype=torch.bfloat16,
    device_map="auto",
)
instruction = "철수가 20개의 연필을 가지고 있었는데 영희가 절반을 가져가고 민수가 남은 5개를 가져갔으면 철수에게 남은 연필의 갯수는 몇개인가요?"

messages = [
    {"role": "user", "content": f"{instruction}"}
    ]

input_ids = tokenizer.apply_chat_template(
    messages,
    add_generation_prompt=True,
    return_tensors="pt"
).to(model.device)

terminators = [
    tokenizer.convert_tokens_to_ids("<|end_of_text|>"),
    tokenizer.convert_tokens_to_ids("<|eot_id|>")
]

outputs = model.generate(
    input_ids,
    max_new_tokens=1024,
    eos_token_id=terminators,
    do_sample=True,
    temperature=0.6,
    top_p=0.9
)

print(tokenizer.decode(outputs[0][input_ids.shape[-1]:], skip_special_tokens=True))

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


철수가 20개의 연필을 가지고 있었고, 영희가 절반을 가져가면 20 / 2 = 10개의 연필을 남기게 됩니다. 

이제 영희가 가져간 10개의 연필을 더한 민수가 5개의 연필을 가져갔습니다. 

따라서 철수가 남은 연필의 갯수는 10 - 5 = 5개입니다. 

결론적으로, 철수에게 남은 연필의 갯수는 5개입니다.
